In [1]:
import pandas as pd

First of, all the CHEBI-hierarchy df

In [2]:
df_hierarchy = pd.read_csv("chebiHierarchy.tsv", sep="\t")
first_part_link = "http://purl.obolibrary.org/obo/CHEBI_"
df_hierarchy['child'] = df_hierarchy['child'].str.extract(r'CHEBI_(\d+)').astype(int)
df_hierarchy['parent'] = df_hierarchy['parent'].str.extract(r'CHEBI_(\d+)').astype(int)
df_hierarchy

,child,parent
0,10,133004
1,11,133004
2,132893,133004
3,132895,133004
4,16777,133004
...,...,...
279058,9915,27810
279059,994,29044
279060,995,19448
279061,9971,69088


*Method 1*

Take the chebis from TCDB, and kill all that have children

In [21]:
all_parents = set(df_hierarchy["parent"])
all_children = df_hierarchy["child"].unique().tolist()
df_filtered = df_hierarchy[~df_hierarchy["child"].isin(all_parents)]
print(df_filtered)

         child  parent
0           10  133004
1           11  133004
2       132893  133004
3       132895  133004
4        16777  133004
...        ...     ...
279058    9915   27810
279059     994   29044
279060     995   19448
279061    9971   69088
279062    9978   45327

[258948 rows x 2 columns]


In [14]:
all_chebi_children = df_filtered["child"].unique().tolist()
len(all_chebi_children)

187792

Now, all the CHEBI IDs from TCDB

In [17]:
df_tcdb = pd.read_csv("../tcdb_data_combined.csv")
all_chebi_tcdb = df_tcdb["CHEBI IDs"].unique().tolist()
len(all_chebi_tcdb)

1656

Assemble the ones that are in both lists

In [31]:
common_children = [id for id in all_chebi_tcdb if id in all_chebi_children ]
print(common_children)
len(common_children)

[135075, 49807, 42452, 40154, 3508, 45453, 29035, 15361, 28009, 17996, 10545, 31460, 61496, 50668, 46502, 15992, 53369, 30031, 45422, 25512, 50434, 383703, 7565, 75861, 27902, 28644, 16763, 16857, 48081, 641, 28199, 29470, 76531, 30297, 16234, 15740, 49468, 29033, 42630, 18022, 37550, 24636, 35673, 18401, 8075, 31432, 29036, 29103, 2453, 29673, 17272, 17474, 32588, 16301, 16150, 15936, 17237, 17439, 28728, 17051, 30150, 16632, 46477, 7731, 6104, 42478, 42626, 63624, 19065, 4883, 9532, 60903, 69437, 41688, 19274, 9948, 20141, 75283, 64626, 47381, 17775, 35475, 71259, 83947, 17563, 6078, 30839, 29108, 72663, 28148, 40646, 6923, 82863, 3764, 31696, 9150, 132804, 16082, 45744, 18019, 30623, 17191, 39473, 443725, 26675, 6714, 26115, 60311, 16995, 34631, 18095, 16865, 49847, 53218, 350546, 30621, 74422, 4911, 85274, 3020, 8984, 23614, 57925, 60210, 4551, 17549, 29806, 63070, 2637, 43968, 78235, 21803, 9555, 50667, 3815, 132153, 64477, 67119, 16452, 75323, 83164, 17698, 9533, 50613, 10101, 10

590

In [27]:
# That was embarassingly few in common for TCDB and all CHEBIs, so I want to see how many of the CHEBIs are further up in the family three. Hopefully 1066...
common_tcdb_all_children = [id for id in all_chebi_tcdb if (id in all_children) and (id not in common_children)]
len(common_tcdb_all_children)

403

In [30]:
# Well 403 children that also are parents. Now I need to check the parents... Hopefully 1656-590-403=663
all_parents2 = df_hierarchy["parent"].unique().tolist()
common_tcdb_all_parents = [id for id in all_chebi_tcdb if (id in all_parents2) and (id not in common_children) and (id not in common_tcdb_all_children)]
len(common_tcdb_all_parents)

0

*Method 2*

Take the chebis from TCDB, and kill all that have children, if their children are on TCDB too.

In [4]:
df_tcdb = pd.read_csv("../tcdb_data_combined.csv")
all_chebi_tcdb = df_tcdb["CHEBI IDs"].unique().tolist()
all_parents = set(df_hierarchy["parent"])
df_filtered = df_hierarchy[~df_hierarchy["child"].isin(all_parents)]

In [13]:
chebi_tcdb_filtered = [el for el in all_chebi_tcdb if el not in all_parents]
len(chebi_tcdb_filtered)

1253

In [14]:
with open("chebis_filtered.txt", "w") as f:
    f.write(",".join(map(str, chebi_tcdb_filtered)))